In [1]:
import matplotlib.pyplot as plt
# 한글 폰트 지정
plt.rc("font", family = "Malgun Gothic")
# 스타일 서식 train
plt.style.use('ggplot')

# 마이너스 기호 표시 설정
plt.rcParams['axes.unicode_minus'] = False

In [2]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

from tqdm import tqdm
import requests
import pandas as pd
import numpy as np
import time
from matplotlib.dates import DateFormatter
import matplotlib.pyplot as plt

- 추가 데이터 : 2021/10/31~2021/11/07 데이터 가져오기

In [3]:
start_date = datetime.strptime('2021-10-30', '%Y-%m-%d')
end_date = datetime.strptime('2021-11-07', '%Y-%m-%d')

date_range = []
current_date = start_date
while current_date <= end_date:
    res_date = current_date.strftime('%Y%m%d')  # 날짜 형식 포맷 변경
    date_range.append(res_date)
    current_date += relativedelta(days=1)

print(date_range)

['20211030', '20211031', '20211101', '20211102', '20211103', '20211104', '20211105', '20211106', '20211107']


In [4]:
start_end_date = []
for i in range(len(date_range) -1):
    start_end_date.append([date_range[i], date_range[i+1]])

In [6]:
# 공공데이터 API
url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'
key = '''5JZYXZopkKriFyvOfT8qjJHfZz45C5yZ9iRCHCmvVUdU/8uqlzaXRLa6fb6yZCNK0VwjjAf2Wgd4NjaXb8GIkw=='''

dfs = []
for i in tqdm(start_end_date):
    print(i)

    params = {'serviceKey' : key, 
            'pageNo' : '1', 
            'numOfRows' : '999', 
            'dataType' : 'JSON', 
            'dataCd' : 'ASOS', ## 자료분류코드
            'dateCd' : 'HR', ## 날짜분류코드
            'startDt' : str(i[0]), ## 시작일
            'startHh' : '00', ## 시작시간
            'endDt' : str(i[1]), ## 종료일
            'endHh': '00', ## 종료시간
            'stnIds' : '131' ## 청주 지역코드 
            }

    response = requests.get(url, params=params)
    result = response.json()
    df = pd.DataFrame(result['response']['body']['items']['item'])
    dfs.append(df)

week_plus = pd.concat(dfs, axis=0, ignore_index=True)

 12%|█▎        | 1/8 [00:00<00:01,  6.82it/s]

['20211030', '20211031']
['20211031', '20211101']


 38%|███▊      | 3/8 [00:00<00:00,  6.84it/s]

['20211101', '20211102']
['20211102', '20211103']


 62%|██████▎   | 5/8 [00:00<00:00,  7.45it/s]

['20211103', '20211104']
['20211104', '20211105']


 88%|████████▊ | 7/8 [00:00<00:00,  8.15it/s]

['20211105', '20211106']
['20211106', '20211107']


100%|██████████| 8/8 [00:01<00:00,  7.59it/s]


In [7]:
week_plus.tail()

,tm,rnum,stnId,stnNm,ta,taQcflg,rn,rnQcflg,ws,wsQcflg,...,lcsCh,vs,gndSttCd,dmstMtphNo,ts,tsQcflg,m005Te,m01Te,m02Te,m03Te
195,2021-11-06 20:00,21,131,청주,15.4,,,,0.1,,...,,1270,,,11.1,,,,,
196,2021-11-06 21:00,22,131,청주,14.3,,,,0.5,,...,,1100,,,10.5,,,,,
197,2021-11-06 22:00,23,131,청주,13.1,,,,0.3,,...,,1100,,,9.8,,,,,
198,2021-11-06 23:00,24,131,청주,12.4,,,,0.2,,...,,1100,,,9.5,,,,,
199,2021-11-07 00:00,25,131,청주,12.8,,,,1.5,,...,,1301,,,9.3,,,,,


In [9]:
# 데이터프레임 결합
if dfs:
    week_plus = pd.concat(dfs, axis=0, ignore_index=True)
    
    # 필요한 열 선택 및 열 이름 변경
    week_plus_selected = week_plus[['tm', 'ta', 'ws', 'hm', 'rn', 'ss']]
    week_plus_selected.columns = ['date_time', '기온(℃)', '풍속(m/s)', '습도(%)', '강수량(mm)', '일조(hr)']

    # 마지막 행 결과 확인
    print(week_plus_selected.tail())
else:
    print("No data fetched from the API.")

            date_time 기온(℃) 풍속(m/s) 습도(%) 강수량(mm) 일조(hr)
195  2021-11-06 20:00  15.4     0.1    52               
196  2021-11-06 21:00  14.3     0.5    58               
197  2021-11-06 22:00  13.1     0.3    64               
198  2021-11-06 23:00  12.4     0.2    66               
199  2021-11-07 00:00  12.8     1.5    66               


In [10]:
week_plus_selected

,date_time,기온(℃),풍속(m/s),습도(%),강수량(mm),일조(hr)
0,2021-10-30 00:00,12.9,0.4,62,,
1,2021-10-30 01:00,11.8,0.5,68,,
2,2021-10-30 02:00,11.3,0.7,71,,
3,2021-10-30 03:00,11.3,0.5,68,,
4,2021-10-30 04:00,10.8,0.3,69,,
...,...,...,...,...,...,...
195,2021-11-06 20:00,15.4,0.1,52,,
196,2021-11-06 21:00,14.3,0.5,58,,
197,2021-11-06 22:00,13.1,0.3,64,,
198,2021-11-06 23:00,12.4,0.2,66,,


In [12]:
# 2021-11-07 00:00 제거
week_plus_selected = week_plus_selected[week_plus_selected['date_time'] != '2021-11-07 00:00']


# date_time 열의 중복 값 제거 (중복된 행 중 첫 번째 행만 남김)
week_plus_selected.drop_duplicates(subset=['date_time'], inplace=True)

In [15]:
week_plus_selected.shape

(192, 6)

In [16]:
week_plus_selected.replace('', np.nan, inplace=True)
week_plus_selected.replace(' ', np.nan, inplace=True)

# 각 열의 결측치 개수 확인
missing_values = week_plus_selected.isnull().sum()

# 결측치 개수 출력
print(missing_values)

date_time      0
기온(℃)          0
풍속(m/s)        0
습도(%)          0
강수량(mm)      188
일조(hr)        96
dtype: int64


In [17]:
# 강수량 열 제거
week_plus_selected.drop(columns=['강수량(mm)'], inplace=True)

# nan을 0으로 대체
week_plus_selected['일조(hr)'].fillna(0, inplace=True)

print(week_plus_selected)

            date_time 기온(℃) 풍속(m/s) 습도(%) 일조(hr)
0    2021-10-30 00:00  12.9     0.4    62      0
1    2021-10-30 01:00  11.8     0.5    68      0
2    2021-10-30 02:00  11.3     0.7    71      0
3    2021-10-30 03:00  11.3     0.5    68      0
4    2021-10-30 04:00  10.8     0.3    69      0
..                ...   ...     ...   ...    ...
194  2021-11-06 19:00  16.7     0.2    45      0
195  2021-11-06 20:00  15.4     0.1    52      0
196  2021-11-06 21:00  14.3     0.5    58      0
197  2021-11-06 22:00  13.1     0.3    64      0
198  2021-11-06 23:00  12.4     0.2    66      0

[192 rows x 5 columns]


In [18]:
week_plus_selected.info()

<class 'pandas.core.frame.DataFrame'>
Index: 192 entries, 0 to 198
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date_time  192 non-null    object
 1   기온(℃)      192 non-null    object
 2   풍속(m/s)    192 non-null    object
 3   습도(%)      192 non-null    object
 4   일조(hr)     192 non-null    object
dtypes: object(5)
memory usage: 9.0+ KB


In [19]:
# 데이터프레임을 엑셀 파일로 저장
week_plus_selected.to_excel("week_plus.xlsx", index=False)